<a href="https://colab.research.google.com/github/mzwing/AI-related/blob/master/notebooks/stable-diffusion.cpp/Anything_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prepare
!rm -rf sample_data
!mkdir -p Anything-GGUF

In [ ]:
# get original model
!sudo apt-get install aria2 -y
!mkdir -p Anything
!echo -e "https://civitai.com/api/download/models/11164\n out=anything_v10.ckpt\nhttps://civitai.com/api/download/models/47248\n out=anything_v21.safetensors\nhttps://civitai.com/api/download/models/34373\n out=anything_v30.ckpt\nhttps://civitai.com/api/download/models/30163\n out=anything_v50.safetensors\nhttps://civitai.com/api/download/models/90854?type=Model&format=SafeTensor&size=full&fp=fp16\n out=anything_inkBase.safetensors" > download.txt
!aria2c -c -x16 -d Anything --input-file=download.txt
!rm -rf download.txt

In [ ]:
# get compile result (CPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/sd-master-583cc5b/sd-master-583cc5b-bin-linux-avx2-openblas-x64.zip
!unzip sd-master-583cc5b-bin-linux-avx2-openblas-x64.zip -d .
!rm -rf sd-master-583cc5b-bin-linux-avx2-openblas-x64.zip

In [ ]:
# prepare for GPU
!sudo apt-get install nvidia-cuda-toolkit -y

In [ ]:
# get compile result (GPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/sd-master-583cc5b/sd-master-583cc5b-bin-linux-avx2-cublas-cu121-x64.zip
!unzip sd-master-583cc5b-bin-linux-avx2-cublas-cu121-x64.zip -d .
!rm -rf sd-master-583cc5b-bin-linux-avx2-cublas-cu121-x64.zip

In [ ]:
# quantize
import concurrent.futures
import os
from tqdm import tqdm

parameters = [ "f32", "f16", "q8_0", "q5_1", "q5_0", "q4_1", "q4_0" ]
models = [ "anything_v10.ckpt", "anything_v21.safetensors", "anything_v30.ckpt", "anything_v50.safetensors", "anything_inkBase.safetensors" ]

tasks = [(model, param) for model in models for param in parameters]

def run_command(task):
    model, param = task
    model_name = model.split('.')[0]
    os.system(f"./sd -M convert -m Anything/{model} -o Anything-GGUF/{model_name}.{param}.gguf --type {param}")

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    list(tqdm(executor.map(run_command, tasks), total=len(tasks)))

In [ ]:
# test
!./sd -m Anything-GGUF/anything_v21.f16.gguf -p "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k" -s -1 -v -o test.png

In [ ]:
!pip install -U --force-reinstall huggingface_hub[hf_transfer]
!huggingface-cli login
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli upload --repo-type model --commit-message "GGUF model commit (made with stable-diffusion.cpp commit 583cc5b)" Anything-GGUF ./Anything-GGUF